# DS/CMPSC 410 Spring 2022
# Instructor: Professor John Yen
# TA: Rupesh Prajapati 
# LAs: Lily Jakielaszek and Cayla Shan Pun
# Lab 5: Movie Recommendations Using Alternative Least Square
## The goals of this lab are for you to be able to
### - Use Alternating Least Squares (ALS) for recommending movies based on reviews of users
### - Be able to understand the raionale for splitting data into training, validation, and testing.
### - Be able to use MLlib to implement an ALS-based movie recommendation system.
### - Be able to use RDD transformations to calculate training error, validation error, and prediction error of a model.
### - Be able to tune hyper-parameters of the ALS model using a small dataset (in local mode)
### - Be able to store the results of evaluating hyper-parameters
### - Be able to select best hyper-parameters and evaluate the chosen model with testing data
### - Be able to improve the efficiency of iterative processing using persist (and unpersist, if applicable)
### - Be able to use CheckPoint to improve the efficiency of iterative processing using Spark.
### - Be able to debug (in local mode) using Restart Kernel if needed.

## Exercises: 
- Exercise 1: 5 points
- Exercise 2: 10 points
- Exercise 3: 10 points
- Exercise 4: 10 points
- Exercise 5: 10 points
- Exercise 6: 10 points
- Exercise 7: 25 points
- Exercise 8: 15 points
- Exercise 9: 10 points
## Total Points: 105 points

# Due: midnight, Feb 13, 2022

## The first thing we need to do in each Jupyter Notebook running pyspark is to import pyspark first.

In [1]:
import pyspark
import pandas as pd
import numpy as np
import math

### Once we import pyspark, we need to import "SparkContext".  Every spark program needs a SparkContext object
### In order to use Spark SQL on DataFrames, we also need to import SparkSession from PySpark.SQL

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, LongType, IntegerType, FloatType
from pyspark.sql.functions import col, column
from pyspark.sql.functions import expr
from pyspark.sql.functions import split
from pyspark.sql import Row
from pyspark.mllib.recommendation import ALS

## We then create a Spark Session variable (rather than Spark Context) in order to use DataFrame. 
- Note: We temporarily use "local" as the parameter for master in this notebook so that we can test it in ICDS Roar.  However, we need to change "local" to "Yarn" before we submit it to XSEDE to run in cluster mode.

In [3]:
ss=SparkSession.builder.master("local").appName("Lab5 Recommendation Systems").getOrCreate()

In [4]:
ss.sparkContext.setCheckpointDir("~/scratch")

## Exercise 1 (5 points) (a) Add your name below AND (b) replace the path below with the path of your home directory.
## Answer for Exercise 1
- a: Your Name: Haichen Wei

In [5]:
ratings_DF = ss.read.csv("/storage/home/hxw5245/Lab4/ratings_2.csv", header=True, inferSchema=True)

In [6]:
ratings_DF.printSchema()

root
 |-- UserID: integer (nullable = true)
 |-- MovieID: integer (nullable = true)
 |-- Rating: double (nullable = true)
 |-- RatingID: integer (nullable = true)



In [7]:
ratings2_DF = ratings_DF.select("UserID","MovieID","Rating")

In [8]:
ratings2_DF.first()

Row(UserID=1, MovieID=31, Rating=2.5)

In [9]:
ratings2_RDD = ratings2_DF.rdd
ratings2_RDD.take(3)

[Row(UserID=1, MovieID=31, Rating=2.5),
 Row(UserID=1, MovieID=1029, Rating=3.0),
 Row(UserID=1, MovieID=1061, Rating=3.0)]

# 5.1 Split Data into Three Sets: Training Data, Evaluatiion Data, and Testing Data

# Exercise 2 (10 points)
## Complete the code below to split `ratings2_RDD` into three groups: 60% training, 20% validation, and 20% testing.

In [10]:
training_RDD, validation_RDD, test_RDD = ratings2_RDD.randomSplit([3, 1, 1], 521)

## Prepare input (UserID, MovieID) for validation and for testing

In [11]:
training_input_RDD = training_RDD.map(lambda x: (x[0], x[1]) )
validation_input_RDD = validation_RDD.map(lambda x: (x[0], x[1]) ) 
testing_input_RDD = test_RDD.map(lambda x: (x[0], x[1]) )

# 5.2 Construct a Movie Recommendation Model 
## using ALS (from `PySpark.MLlib.recommendation` module) and training data

In [12]:
model = ALS.train(training_RDD, 4, seed=41, iterations=30, lambda_=0.1)

# Exercise 3 (10 points)
## Complete the code below to transform the model output of training data into the format of `( (<UserID> <MovieID>), <predictedRating> )` so that we can later join it with actual Rating for computing RMS.

In [13]:
training_prediction_RDD = model.predictAll(training_input_RDD).map(lambda x: ( (x[0], x[1]), x[2] ) )

In [14]:
training_prediction_RDD.take(3)

[((599, 69069), 3.925916741285466),
 ((270, 81132), 3.9015987051642433),
 ((390, 667), 2.33796273224989)]

In [15]:
training_evaluation_RDD = training_RDD.map(lambda y: ( (y[0], y[1]), y[2]) ).join(training_prediction_RDD)

In [16]:
training_evaluation_RDD.take(3)

[((1, 31), (2.5, 2.59967650141975)),
 ((1, 1029), (3.0, 2.697573645098246)),
 ((1, 1061), (3.0, 2.7537082775211985))]

In [17]:
training_error = math.sqrt(training_evaluation_RDD.map(lambda z: (z[1][0] - z[1][1])**2).mean())

In [18]:
print(training_error)

0.6498690270748771


In [19]:
validation_prediction_RDD = model.predictAll(validation_input_RDD).map(lambda x: ( (x[0], x[1]), x[2] ) )

In [20]:
validation_prediction_RDD.take(3)

[((402, 69069), 3.2310862470123247),
 ((624, 69069), 2.210174197884259),
 ((213, 44828), 2.174816531472955)]

# Exercise 4 (10 points)
## Complete the code below to join `validation_RDD` (after transforming it into the same key value pair format of Exercise 3, and `validation_prediction_RDD` to prepare for RMS error calculation.

In [27]:
validation_evaluation_RDD = validation_RDD.map(lambda y: ( (y[0], y[1]), y[2] )).join(validation_prediction_RDD)

In [28]:
validation_evaluation_RDD.take(3)

[((1, 1129), (2.0, 2.3653280077890746)),
 ((1, 1953), (4.0, 2.789249674020755)),
 ((1, 2105), (4.0, 2.1729105277828893))]

# Exercise 5 (10 points)
## Complete the code below to calculate RMS error for validation data.

In [29]:
validation_error = math.sqrt(validation_evaluation_RDD.map(lambda z: (z[1][0] - z[1][1])**2).mean())

In [30]:
print(validation_error)

0.9400613190737446


# Exercise 6 (10 points)
## Complete the code below that computes the RMS errors of training data and validation data for three values of rank (k = 4, 7, and 10) in ALS model (for regularization hyperparameter = 0.1, number of iterations 30).

In [31]:
index =0 
# initialize lowest_error
lowest_validation_error = float('inf')
lowest_training_error = float('inf')
# Set up the possible hyperparameter values to be evaluated
iterations = 30
regularization = 0.1
rank_list = [4, 7, 10]
for k in rank_list:
    seed = 43
    # Construct a recommendation model using a set of hyper-parameter values and training data
    model = ALS.train(training_RDD, k, seed=seed, iterations=iterations, lambda_=regularization)
    training_prediction_RDD = model.predictAll(training_input_RDD).map(lambda x: ( (x[0], x[1]), x[2] ) )
    training_evaluation_RDD = training_RDD.map(lambda y: ( (y[0], y[1]), y[2] ) ).join(training_prediction_RDD)
    training_error = math.sqrt(training_evaluation_RDD.map(lambda z: (z[1][0] - z[1][1])**2).mean())
    # Evaluate the model using evalution data
    # map the output into ( (userID, movieID), rating ) so that we can join with actual evaluation data
    # using (userID, movieID) as keys.
    validation_prediction_RDD= model.predictAll(validation_input_RDD).map(lambda x: ( (x[0], x[1]), x[2] )   )
    validation_evaluation_RDD = validation_RDD.map(lambda y: ( (y[0], y[1]), y[2] )  ).join(validation_prediction_RDD)
    # Calculate RMS error between the actual rating and predicted rating for (userID, movieID) pairs in validation dataset
    validation_error = math.sqrt(validation_evaluation_RDD.map(lambda z: (z[1][0] - z[1][1])**2).mean())
    # Save the error as a row in a pandas DataFrame
    # hyperparams_eval_df.loc[index] = [k, regularization, iterations, training_error, validation_error, float('inf')]
    index = index + 1
    print(' rank is ', k, ', Training Error = ', training_error, ', Validation Error =', validation_error)
    # Check whether the current error is the lowest
    if validation_error < lowest_validation_error:
        best_k = k
        best_validation_index = index - 1 
        lowest_validation_error = validation_error
    if training_error < lowest_training_error:
        best_training_k = k
        best_training_index = index - 1
        lowest_training_error = training_error

 rank is  4 , Training Error =  0.6495805650417658 , Validation Error = 0.9337770394103238
 rank is  7 , Training Error =  0.573574429116544 , Validation Error = 0.9473097548982903
 rank is  10 , Training Error =  0.5265304336062925 , Validation Error = 0.9455620990645692


# 5.3 Hyperparameter Tuning
## Iterate through all possible combination of a set of values for three hyperparameters for ALS Recommendation Model:
- rank (k)
- regularization
- iterations 
## Each hyperparameter value combination is used to construct an ALS recommendation model using training data, but evaluate using Evaluation Data
## The evaluation results are saved in a Pandas DataFrame 
``
hyperparams_eval_df
``
## The best hyperprameter value combination is stored in 4 variables
``
best_k, best_regularization, best_iterations, and lowest_validation_error
``

# Exercise 7 (25 points) 
## Complete the code below to iterate through the following set of hyperparameters to create and evaluate ALS recommendation models:
- iterations : 15, 30
- regularization: 0.1, 0.2
- rank: 4, 7, 10

In [32]:
## Initialize a Pandas DataFrame to store evaluation results of all combination of hyper-parameter settings
hyperparams_eval_df = pd.DataFrame( columns = ['k', 'regularization', 'iterations', 'validation RMS', 'testing RMS'] )
# initialize index to the hyperparam_eval_df to 0
index =0 
# initialize lowest_error
lowest_validation_error = float('inf')
# Set up the possible hyperparameter values to be evaluated
iterations_list = [15, 30]
regularization_list = [0.1, 0.2]
rank_list = [4, 7, 10]
for k in rank_list:
    for regularization in regularization_list:
        for iterations in iterations_list:
            seed = 37
            # Construct a recommendation model using a set of hyper-parameter values and training data
            model = ALS.train(training_RDD, k, seed=seed, iterations=iterations, lambda_=regularization)
            # Evaluate the model using evalution data
            # map the output into ( (userID, movieID), rating ) so that we can join with actual evaluation data
            # using (userID, movieID) as keys.
            validation_prediction_RDD= model.predictAll(validation_input_RDD).map(lambda x: ( (x[0], x[1]), x[2])   )
            validation_evaluation_RDD = validation_RDD.map(lambda y: ( (y[0], y[1]), y[2]) ).join(validation_prediction_RDD)
            # Calculate RMS error between the actual rating and predicted rating for (userID, movieID) pairs in validation dataset
            validation_error = math.sqrt(validation_evaluation_RDD.map(lambda z: (z[1][0] - z[1][1])**2).mean())
            # Save the error as a row in a pandas DataFrame
            hyperparams_eval_df.loc[index] = [k, regularization, iterations, validation_error, float('inf')]
            index = index + 1
            # Check whether the current error is the lowest
            if validation_error < lowest_validation_error:
                best_k = k
                best_regularization = regularization
                best_iterations = iterations
                best_index = index - 1
                lowest_validation_error = validation_error
print('The best rank k is ', best_k, ', regularization = ', best_regularization, ', iterations = ',\
      best_iterations, '. Validation Error =', lowest_validation_error)

The best rank k is  7 , regularization =  0.2 , iterations =  30 . Validation Error = 0.9181386561528002


# Use Testing Data to Evaluate the Model built using the Best Hyperparameters                

# 5.4 Evaluate the best hyperparameter combination using testing data
## If the error between rating prediction and actual rating for (userID, movie ID) pairs in the training data is comparable to the error of validation data, our model passes the test.

# Exercise 8 (15 points)
- (a) Complete the code below to evaluate the best hyperparameter combinations using testing data. (10 point)
- (b) Does your model pass the test of testing data? Explain your answer in the Markdown cell below. (5 points)

In [33]:
seed = 37
model = ALS.train(training_RDD, best_k, seed=seed, iterations=best_iterations, lambda_=best_regularization)
testing_prediction_RDD=model.predictAll(testing_input_RDD).map(lambda x: ((x[0], x[1]), x[2]))
testing_evaluation_RDD= test_RDD.map(lambda x: ((x[0], x[1]), x[2])).join(testing_prediction_RDD)
testing_error = math.sqrt(testing_evaluation_RDD.map(lambda x: (x[1][0] - x[1][1])**2).mean())
print('The Testing Error for rank k =', best_k, ' regularization = ', best_regularization, ', iterations = ', \
      best_iterations, ' is : ', testing_error)

The Testing Error for rank k = 7  regularization =  0.2 , iterations =  30  is :  0.9181095896148572


# Answer to Exercise 8: 
- (b) Yes, the model pass the test, both best rank k is 7, regularaization is 0.2, iterations is 30, and the testing error is close to the validation error.

In [34]:
print(best_index)

7


In [35]:
# Store the Testing RMS in the DataFrame
hyperparams_eval_df.loc[best_index]=[best_k, best_regularization, best_iterations, lowest_validation_error, testing_error]

In [36]:
schema3= StructType([ StructField("k", FloatType(), True), \
                      StructField("regularization", FloatType(), True ), \
                      StructField("iterations", FloatType(), True), \
                      StructField("Validation RMS", FloatType(), True), \
                      StructField("Testing RMS", FloatType(), True) \
                    ])

## Convert the pandas DataFrame that stores validation errors of all hyperparameters and the testing error for the best model to Spark DataFrame


In [37]:
HyperParams_Tuning_DF = ss.createDataFrame(hyperparams_eval_df, schema3)

# Exercise 9 (10 points)
## Modify the output path so that your output results can be saved in a directory.

In [38]:
output_path = "/storage/home/hxw5245/Lab5/Lab5ALSHyperParamsTuning"
HyperParams_Tuning_DF.rdd.saveAsTextFile(output_path)

In [39]:
ss.stop()